# Inverse dynamics
Each manoeuvring model has some hydrodynamic functions $X_D(u,v,r,\delta,thrust)$, $Y_D(u,v,r,\delta,thrust)$, $N_D(u,v,r,\delta,thrust)$ that are defined as polynomials. 

The hydrodynamic derivatives in these polynomials can be identified with **force regression** of measured forces and moments. The measured forces and moments are usually taken from captive model tests or Virtual Captive Tests (VCT) being the virtual version of captive tests calculated with Computational Fluid Dynamics (CFD). 

When the ship or ship model is free in all degrees of freedome, as in the present model tests, only motions can be observed however. The forces and moments that generated the motions needs to be estimated by solving the inverse dynamics problem and doing a **motion regression**. 

In [1]:
# %load imports.py
%load_ext autoreload
%autoreload 2
%reload_kedro
%config Completer.use_jedi = False  ## (To fix autocomplete)

import pandas as pd
from src.models.vmm import ModelSimulator
import matplotlib.pyplot as plt
from src.visualization.plot import track_plots, plot, captive_plot
import kedro
import numpy as np
import os.path
import anyconfig

import matplotlib
matplotlib.rcParams["figure.figsize"] = (15,4)

from myst_nb import glue
from src.symbols import *
import src.symbols as symbols
from src.system_equations import *

from IPython.display import display, Math, Latex, Markdown
from sympy.physics.vector.printing import vpprint, vlatex

from src.parameters import df_parameters
p = df_parameters["symbol"]

# Read configs:
conf_path = os.path.join("../../conf/base/")
runs_globals_path = os.path.join(
    conf_path,
    "runs_globals.yml",
)

runs_globals = anyconfig.load(runs_globals_path)
model_test_ids = runs_globals["model_test_ids"]

join_globals_path = os.path.join(
    conf_path,
    "join_globals.yml",
)

joins = runs_globals["joins"]
join_runs_dict = anyconfig.load(join_globals_path)

globals_path = os.path.join(
    conf_path,
    "globals.yml",
)
global_variables = anyconfig.load(globals_path)



vmms = global_variables["vmms"]
only_joined = global_variables[
    "only_joined"
]  # (regress/predict with only models from joined runs)S

2022-02-01 08:43:29,853 - kedro.framework.session.store - INFO - `read()` not implemented for `SQLiteStore`. Assuming empty store.
2022-02-01 08:43:30,844 - root - INFO - ** Kedro project wPCC_pipeline
2022-02-01 08:43:30,844 - root - INFO - Defined global variable `context`, `session`, `catalog` and `pipelines`
2022-02-01 08:43:30,859 - root - INFO - Registered line magic `run_viz`


e:\dev\wpcc\src\symbols.py:97: DeprecationWarning: invalid escape sequence \d
  x_0, x_01d = sp.symbols("x_0 \dot{x_0}")
e:\dev\wpcc\src\symbols.py:98: DeprecationWarning: invalid escape sequence \d
  y_0, y_01d = sp.symbols("y_0 \dot{y_0}")
e:\dev\wpcc\src\symbols.py:99: DeprecationWarning: invalid escape sequence \P
  psi, psi1d = sp.symbols("\Psi \dot{\Psi}")
E:\dev\wPCC_pipeline\venv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
E:\dev\wPCC_pipeline\venv\lib\site-packages\mlflow\types\schema.py:49: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.

In [2]:
from src.models.vmm_martin import martins_model
from sympy import MatrixSymbol
from src.models.regression import MotionRegression

In [3]:
id = 22774
df = catalog.load(f"{ id }.data_ek_smooth")
added_masses = catalog.load("added_masses")
ship_parameters = catalog.load("ship_data")

regression = MotionRegression(vmm=martins_model, 
                              data=df, 
                              added_masses=added_masses, 
                              ship_parameters=ship_parameters)

2022-02-01 08:43:37,587 - kedro.io.data_catalog - INFO - Loading data from `22774.data_ek_smooth` (CSVDataSet)...
2022-02-01 08:43:37,633 - kedro.io.data_catalog - INFO - Loading data from `added_masses` (YAMLDataSet)...
2022-02-01 08:43:37,635 - kedro.io.data_catalog - INFO - Loading data from `ship_data` (YAMLDataSet)...


e:\dev\wpcc\src\models\regression.py:103: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self.exclude_parameters = pd.Series(exclude_parameters)
<string>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


Finding the the hydrodynamic derivatives can be defined as a linear regression problem:

$$ y = X \beta + \epsilon $$

for the case with $n$ data points an $N$ features $y$ is an $[n * 1]$ output vector (label), $X$ is $[n * N]$ input matrix (features) and $\beta$ is a $[N * 1]$ vector with the regressed coefficients. 

A linear regression can be defined for each of the degrees of freedomes, giving three regressions with feature matrices, replacing $X$ with : $X_X$, $X_Y$, $X_N$ and coefficient vectors, replacing $\beta$ with: $A_{coeff}$, $B_{coeff}$, $C_{coeff}$.


In [24]:
epsilon = sp.symbols('epsilon')

eq_regression_u1d = sp.Eq(u1d, sp.UnevaluatedExpr(X_X*A_coeff) + epsilon)
eq_regression_v1d = sp.Eq(v1d, sp.UnevaluatedExpr(X_Y*B_coeff) + epsilon)
eq_regression_r1d = sp.Eq(r1d, sp.UnevaluatedExpr(X_N*C_coeff) + epsilon)
display(eq_regression_u1d)
display(eq_regression_v1d)
display(eq_regression_r1d)


Eq(\dot{u}, epsilon + X_X*A_coeff)

Eq(\dot{v}, epsilon + X_Y*B_coeff)

Eq(\dot{r}, epsilon + X_N*C_coeff)

The right hand side of the system equation can also be expressed in matrix form:

In [62]:
X_qs_ = MatrixSymbol("X_qs_", N, 1)
eq_Xqs =  sp.Eq(sp.UnevaluatedExpr(X_X*X_qs_), eq_system.rhs.doit()[0])

Y_qs_ = MatrixSymbol("Y_qs_", N, 1)
eq_Yqs =  sp.Eq(sp.UnevaluatedExpr(X_Y*Y_qs_), eq_system.rhs.doit()[1])

N_qs_ = MatrixSymbol("N_qs_", N, 1)
eq_Nqs =  sp.Eq(sp.UnevaluatedExpr(X_N*N_qs_), eq_system.rhs.doit()[2])

display(eq_Xqs)
display(eq_Yqs)
display(eq_Nqs)


Eq(X_X*X_qs_, m*r**2*x_G + m*r*v + X_D(u, v, r, delta, thrust))

Eq(X_Y*Y_qs_, -m*r*u + Y_D(u, v, r, delta, thrust))

Eq(X_N*N_qs_, -m*r*u*x_G + N_D(u, v, r, delta, thrust))

The system equation can now be expressed in pure matrix form:

In [86]:
eq_decoupling =  eq_system.subs([
    (u1d,(X_X*A_coeff).transpose()),
    (v1d,(X_Y*B_coeff).transpose()),
    (r1d,(X_N*C_coeff).transpose()),
    
    (eq_Xqs.rhs,eq_Xqs.lhs.doit().transpose()),
    (eq_Yqs.rhs,eq_Yqs.lhs.doit().transpose()),
    (eq_Nqs.rhs,eq_Nqs.lhs.doit().transpose()),
    
])

eq_decoupling

Eq(Matrix([
[-X_{\dot{u}} + m,                    0,                    0],
[               0,     -Y_{\dot{v}} + m, -Y_{\dot{r}} + m*x_G],
[               0, -N_{\dot{v}} + m*x_G,    I_z - N_{\dot{r}}]])*Matrix([
[A_coeff.T*X_X.T],
[B_coeff.T*X_Y.T],
[C_coeff.T*X_N.T]]), Matrix([
[X_qs_.T*X_X.T],
[Y_qs_.T*X_Y.T],
[N_qs_.T*X_N.T]]))

And the feature matrices dissapear so that the coefficient vectors can be calculates as:

In [85]:
coeffs = sp.matrices.MutableDenseMatrix([A_coeff.transpose(), 
                                         B_coeff.transpose(), 
                                         C_coeff.transpose()])

qs = sp.matrices.MutableDenseMatrix([X_qs_.transpose(), 
                                         Y_qs_.transpose(), 
                                         N_qs_.transpose()])

sp.Eq(sp.UnevaluatedExpr(qs), sp.UnevaluatedExpr(A)*sp.UnevaluatedExpr(coeffs))

Eq(Matrix([
[X_qs_.T],
[Y_qs_.T],
[N_qs_.T]]), Matrix([
[-X_{\dot{u}} + m,                    0,                    0],
[               0,     -Y_{\dot{v}} + m, -Y_{\dot{r}} + m*x_G],
[               0, -N_{\dot{v}} + m*x_G,    I_z - N_{\dot{r}}]])*Matrix([
[A_coeff.T],
[B_coeff.T],
[C_coeff.T]]))

Note that the decoupled and regressed parameters now also contain the contributions from the coriolis/centrepetal forces, which need to be substracted so that the parameters contain only hydrodyamic contributions. For instance in the X-equation $m \cdot x_G$ needs to be substracted from $X_{rr}$ and $m$ needs to be subtracted from the $X_{vr}$ derivative.

It has been confirmed that regression of the inverse dynamics can be solved with Ordinary Least Squares (OLS) to identidy the hydrodynamic derivatives in a VMM. Identifying the derivatives on simulated data gives a perfect match between the regressed parameters and the real model. This is the case when the model captures the physics perfectly (when there is no process noise $w$) and no measurement noise $\epsilon$. 

For real data from model tests or full scale ships this is of course never the case. Handling measurement errors and the fact the VMM used will be a good approximation of the real physics, at best, introduces a lot of challenges. Usually only the position and heading of the ships are measured, which means that the higher order states of velocities and accelerations are usually unknown. This introduces another challenge. Velocities and accelerations from noisy position measurements can be estimated using Extended Kalman Filter and Extended Kalman Smoother which will be introduced in the next section.